In [1]:
import pandas as pd
from random import sample
from datetime import datetime
from ast import literal_eval
from math import nan
from json import loads

pd.options.display.max_columns = None

In [2]:
df_calendar = pd.read_csv('data/raw/calendar.csv.gz')
df_listings = pd.read_csv('data/raw/listings.csv.gz')
df_reviews = pd.read_csv('data/raw/reviews.csv.gz')

In [3]:
with open('data/mapeamento_bairro_subprefeitura.json', 'r') as f:
    map_bairro_subpref = loads(f.read())

## Define funções de limpeza

In [4]:
def parse_money(s):
    return float(s.replace('$', '').replace(',', ''))

In [5]:
def parse_bool(s):
    return s == 't' if s in {'t', 'f'} else None

In [6]:
def parse_date(s, form='%Y-%m-%d'):
    if isinstance(s, str):
        return datetime.strptime(s, form)
    else:
        return None

In [7]:
def parse_perc(s):
    return float(s.replace('%', ''))/100 if isinstance(s, str) else s

In [8]:
def parse_list(df, col):
    pool = set()
    for s in df[col]:
        if s != 'None':
            pool = pool.union(literal_eval(s))
    pool = [(f'{col}_{elem}', f"'{elem}'") for elem in sorted(pool)]
    for dummie_col, elem in pool:
        df[dummie_col] = df[col].str.contains(elem)
    return df
        

In [9]:
def tokenize(df, col):
    null_marker = 'NaN'
    options = sorted(df[col].fillna(null_marker).unique())
    if null_marker in options:
        options.remove(null_marker)
    for opt in options:
        df[f'{col}_{opt}'] = df[col] == opt
    return df

In [10]:
def parse_bathroom(df):
    df['number_of_bathrooms'] = df.bathrooms_text.str.replace('\D','').apply(lambda x: nan if x == '' else x).astype(float)
    df['bathroom_description'] = df.bathrooms_text.str.replace('\d','').str.replace('.', '').str.strip()
    df = tokenize(df, 'bathroom_description')
    return df

In [11]:
def separa_datasets(df, id_column, id_list):
    return df[df[id_column].isin(id_list)].copy().reset_index(drop=True)

## Limpa Datasets

In [12]:
df_calendar['price'] = df_calendar['price'].apply(parse_money)
df_calendar['adjusted_price'] = df_calendar['adjusted_price'].apply(parse_money)
df_calendar['available'] = df_calendar['available'].apply(parse_bool)
df_calendar['date'] = df_calendar['date'].apply(parse_date)

In [13]:
df_reviews['date'] = df_reviews['date'].apply(parse_date)

In [14]:
df_listings['last_scraped'] = df_listings['last_scraped'].apply(parse_date)
df_listings['host_since'] = df_listings['host_since'].apply(parse_date)
df_listings = tokenize(df_listings, 'host_response_time')
df_listings['host_response_rate'] = df_listings['host_response_rate'].apply(parse_perc)
df_listings['host_acceptance_rate'] = df_listings['host_acceptance_rate'].apply(parse_perc)
df_listings['host_is_superhost'] = df_listings['host_is_superhost'].apply(parse_bool)
df_listings = parse_list(df_listings, 'host_verifications')
df_listings['host_has_profile_pic'] = df_listings['host_has_profile_pic'].apply(parse_bool)
df_listings['host_identity_verified'] = df_listings['host_identity_verified'].apply(parse_bool)
df_listings = tokenize(df_listings, 'neighbourhood_cleansed')
df_listings = tokenize(df_listings, 'property_type')
df_listings = tokenize(df_listings, 'room_type')
df_listings = parse_bathroom(df_listings)
df_listings = parse_list(df_listings, 'amenities')
df_listings['price'] = df_listings['price'].apply(parse_money)
df_listings['has_availability'] = df_listings['has_availability'].apply(parse_bool)
df_listings['calendar_last_scraped'] = df_listings['calendar_last_scraped'].apply(parse_date)
df_listings['first_review'] = df_listings['first_review'].apply(parse_date)
df_listings['last_review'] = df_listings['last_review'].apply(parse_date)
df_listings['instant_bookable'] = df_listings['instant_bookable'].apply(parse_bool)
df_listings['subprefeitura'] = df_listings['neighbourhood_cleansed'].apply(lambda x: map_bairro_subpref[x])
df_listings = tokenize(df_listings, 'subprefeitura')

<ipython-input-10-fe88eb267c4c>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['number_of_bathrooms'] = df.bathrooms_text.str.replace('\D','').apply(lambda x: nan if x == '' else x).astype(float)
<ipython-input-10-fe88eb267c4c>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['bathroom_description'] = df.bathrooms_text.str.replace('\d','').str.replace('.', '').str.strip()
<ipython-input-10-fe88eb267c4c>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['bathroom_description'] = df.bathrooms_text.str.replace('\d','').str.replace('.', '').str.strip()
/Users/Jonas/Library/Python/3.8/lib/python/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extr

## Separa datasets

Usaremos k-fold para selecionar metaparâmetros, então separarei 3000 anúncios para avaliar a qualidade do modelo final

In [15]:
ids = list(df_listings.id)

test_size = 3000
test_index = sample(range(len(ids)), test_size)

test_ids = {listing_id for index, listing_id in enumerate(ids) if index in test_index}
training_ids = {listing_id for index, listing_id in enumerate(ids) if index not in test_index}

In [16]:
test_listing      = separa_datasets(df_listings, 'id', test_ids)
training_listing  = separa_datasets(df_listings, 'id', training_ids)

test_calendar     = separa_datasets(df_calendar, 'listing_id', test_ids)
training_calendar = separa_datasets(df_calendar, 'listing_id', training_ids)

test_reviews      = separa_datasets(df_reviews, 'listing_id', test_ids)
training_reviews  = separa_datasets(df_reviews, 'listing_id', training_ids)

In [17]:
test_listing.to_parquet('data/trusted/test/listing.parquet', index=False)
training_listing.to_parquet('data/trusted/training/listing.parquet', index=False)

test_calendar.to_parquet('data/trusted/test/calendar.parquet', index=False)
training_calendar.to_parquet('data/trusted/training/calendar.parquet', index=False)

test_reviews.to_parquet('data/trusted/test/reviews.parquet', index=False)
training_reviews.to_parquet('data/trusted/training/reviews.parquet', index=False)